# Recommending Without User Data: Content Filtering With NLP and Cosine Similarity

Using data previously cleaned, I'm going to use several methods of NLP along with cosine similarity to make a simple, initial movie recommendation system. This sytem can be used to recommend movies when users are brand new. There are several ways this can be used:

1. The user can search for similar movies to a movie they like when a user first signs up with the app or service, but movies will be very closely related (Iron Man, Iron Man II, Iron Man III, etc)
2. The app can use this feature to suggest similar movies for the User to initially rate. I.e if the user says they like Toy Story, movies with a high cosine similarity can be the next suggestions for a user to rate the movie. 
3. This method can be used to group similar movies together into recommended "pods". For example, "Japanese animation featuring strong female leads" might include Spirited Away and Princess Mononoke.


### Features Used For Recommendations

* **Rated** - The movie's rating
* **Year** - The Release Year
* **Genre** - The movie's genres
* **Writer** - Who wrote the screenplay
* **Actors** - Significant actors in the movie
* **Director** - who directed the movies
* **Key words** - Key words taken from the plot using [nltk_rake](https://pypi.org/project/rake-nltk/)

In [2]:
import pandas as pd
import numpy as np

import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# 1. Bag of Words and TF-IDF

In [29]:
movies = pd.read_csv('/Users/user/final-capstone/inprogress3.csv')
print(movies.shape)
movies.head()

(284303, 25)


,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Toy Story,1995,G,22-Nov-95,81 min,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A cowboy doll is profoundly threatened and jea...,...,95.0,8.3,"788,709",tt0114709,movie,20-Mar-01,NaN,Buena Vista,http://www.disney.com/ToyStory,True
1,Jumanji,1995,PG,15-Dec-95,104 min,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,...,39.0,7.0,"267,973",tt0113497,movie,25-Jan-00,NaN,Sony Pictures Home Entertainment,NaN,True
2,Grumpier Old Men,1995,PG-13,22-Dec-95,101 min,"Comedy, Romance",Howard Deutch,"Mark Steven Johnson (characters), Mark Steven ...","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",John and Max resolve to save their beloved bai...,...,46.0,6.6,"22,329",tt0113228,movie,18-Nov-97,NaN,Warner Home Video,NaN,True
3,Waiting to Exhale,1995,R,22-Dec-95,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...","Based on Terry McMillan's novel, this film fol...",...,NaN,5.8,"8,640",tt0114885,movie,6-Mar-01,NaN,Twentieth Century Fox Home Entertainment,NaN,True
4,Father of the Bride Part II,1995,PG,8-Dec-95,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...",George Banks must deal not only with the pregn...,...,49.0,6.0,"31,241",tt0113041,movie,9-May-00,NaN,Disney,NaN,True


In [30]:
movies = movies.drop(['website','response','type','ratings','metascore','dvd','box_office'],axis=1)

# Drop duplicate movies from the dataframe
movies = movies.drop_duplicates(subset='plot', keep= 'first')

# Drop TV shows from the dataframe
ratings = ['TV-14', 'TV-PG', 'TV-MA', 'TV-G', 'TV-Y', 'TV-13', 'TV-Y7']

for rating in ratings:
    movies = movies[movies.rated != rating]
    
    
# Drop a few other misc TV shows that snuck through
titles = ['The Mahabharata', 'Centennial', 'Dominion Tank Police', 'Children of Dune', 'Helen of Troy']

for title in titles:
    movies = movies[movies['title'] != title]

movies['year'] = movies['year'].str.strip('–')
movies['year'] = pd.to_numeric(movies['year'])
    
# Due to computational power on my machine, only looking at movies later than 1985
movies = movies[movies['year'] >= 1987]

# Dropping movies that have fewer than 100 ratings on imdb only so that we can have computational power

movies['imdb_votes'] = movies['imdb_votes'].str.replace(",","")
movies['imdb_votes'] = pd.to_numeric(movies['imdb_votes'])
movies = movies[movies['imdb_votes'] > 10]
print(movies.shape)

(72585, 18)


In [31]:
movies.columns

Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'imdb_rating', 'imdb_votes', 'imdb_id', 'production'],
      dtype='object')

In [32]:
vote_counts = movies[movies['imdb_votes'].notnull()]['imdb_votes'].astype('int')
vote_averages = movies[movies['imdb_rating'].notnull()]['imdb_rating'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['imdb_votes']+1 # added +1 - Dan
    R = x['imdb_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

movies['weighted_rating'] = movies.apply(weighted_rating, axis=1)

In [33]:
movies.shape

(72585, 19)

In [34]:
# Test 1: IMDB rating above 4.5, 63,373 movies
# Test 2: IMDB rating above 5, 58,478 movies

movies = movies[movies['imdb_rating'] > 5.5]
movies.shape

(51520, 19)

In [207]:
#  merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
#for index, row in movies.iterrows():
#    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
#    row['director'] = ''.join(row['director']).lower()

In [36]:
movies = movies[['title','weighted_rating','imdb_rating','imdb_id','rated','writer','genre','director','actors','plot','language','country', 'year']]

for column in movies:
    movies[column] = movies[column].apply(lambda x: str(x))
    
# discarding the commas between the actors' full names
movies['actors'] = movies['actors'].map(lambda x: x.split(','))

# putting the genres in a list of words
movies['genre'] = movies['genre'].map(lambda x: x.lower().split(','))

movies['director'] = movies['director'].map(lambda x: x.split(','))


# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in movies.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = [x.lower().replace(' ','') for x in row['director']]


# Getting rid of information between () in writers
movies['writer'] = movies['writer'].str.replace(r"\(.*\)","")

# discarding the commas between writer's names
movies['writer'] = movies['writer'].map(lambda x: x.split(','))

# putting the countries in a list of words
movies['country'] = movies['country'].map(lambda x: x.lower().split(','))

# putting the languages in a list of words
movies['language'] = movies['language'].map(lambda x: x.lower().split(','))

movies['rated'] = movies['rated'].str.replace("-","")

movies['year'] = movies['year'].map(lambda x: x.split(','))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in movies.iterrows():
    row['writer'] = [x.lower().replace(' ','') for x in row['writer']]


In [37]:
movies.head()

,title,weighted_rating,imdb_rating,imdb_id,rated,writer,genre,director,actors,plot,language,country,year
0,Toy Story,8.296249489578639,8.3,tt0114709,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",A cowboy doll is profoundly threatened and jea...,[english],[usa],[1995.0]
1,Jumanji,6.994508880344042,7.0,tt0113497,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...",When two kids find and play a magical board ga...,"[english, french]",[usa],[1995.0]
2,Grumpier Old Men,6.556503572029107,6.6,tt0113228,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...",John and Max resolve to save their beloved bai...,"[english, italian, german]",[usa],[1995.0]
3,Waiting to Exhale,5.789027071723114,5.8,tt0114885,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...","Based on Terry McMillan's novel, this film fol...",[english],[usa],[1995.0]
4,Father of the Bride Part II,5.989632282691058,6.0,tt0113041,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",George Banks must deal not only with the pregn...,[english],[usa],[1995.0]


In [25]:
temp = movies[['title','weighted_rating','imdb_id']]

In [38]:
movies = movies.drop(['weighted_rating','imdb_id','imdb_rating'],axis=1)

In [39]:
# initializing the new column
movies['Key_words'] = ""

for index, row in movies.iterrows():
    plot = row['plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
movies.drop(columns = ['plot'], inplace = True)

In [40]:
movies.head()

,title,rated,writer,genre,director,actors,language,country,year,Key_words
0,Toy Story,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995.0],"[cowboy, doll, boy, profoundly, threatened, ne..."
1,Jumanji,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995.0],"[magical, board, game, decades, finishing, pla..."
2,Grumpier Old Men,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995.0],"[max, resolve, new, female, owner, catches, at..."
3,Waiting to Exhale,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995.0],"[novel, film, follows, four, terry, mcmillan, ..."
4,Father of the Bride Part II,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995.0],"[unexpected, pregnancy, daughter, wife, also, ..."


In [41]:
movies.set_index('title', inplace = True)
movies.head()

,rated,writer,genre,director,actors,language,country,year,Key_words
title,,,,,,,,,
Toy Story,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995.0],"[cowboy, doll, boy, profoundly, threatened, ne..."
Jumanji,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995.0],"[magical, board, game, decades, finishing, pla..."
Grumpier Old Men,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995.0],"[max, resolve, new, female, owner, catches, at..."
Waiting to Exhale,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995.0],"[novel, film, follows, four, terry, mcmillan, ..."
Father of the Bride Part II,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995.0],"[unexpected, pregnancy, daughter, wife, also, ..."


In [42]:
movies['bag_of_words'] = ''
columns = movies.columns
for index, row in movies.iterrows():
    words = ''
    for col in columns:
        if col == 'rated':
            words = words.join(row[col]) + ' '
        elif col == 'imdb_rating':
            None
        else:
            words = words + ' '.join(row[col])+ ' '
#        else:
#            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
#df.drop(columns = [col for col in df.columns if col!= 'bag_of_words' or 'title'], inplace = True)

In [43]:
movies.head()

,rated,writer,genre,director,actors,language,country,year,Key_words,bag_of_words
title,,,,,,,,,,
Toy Story,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995.0],"[cowboy, doll, boy, profoundly, threatened, ne...",G johnlasseter animation adventure comedy f...
Jumanji,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995.0],"[magical, board, game, decades, finishing, pla...",PG jonathanhensleigh adventure comedy family...
Grumpier Old Men,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995.0],"[max, resolve, new, female, owner, catches, at...",PG13 markstevenjohnson markstevenjohnson comed...
Waiting to Exhale,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995.0],"[novel, film, follows, four, terry, mcmillan, ...",R terrymcmillan comedy drama romance forestw...
Father of the Bride Part II,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995.0],"[unexpected, pregnancy, daughter, wife, also, ...",PG alberthackett comedy family romance charl...


In [44]:
movies['bag_of_words'][0]

'G johnlasseter animation  adventure  comedy  family  fantasy johnlasseter tomhanks timallen donrickles jimvarney english usa 1995.0 cowboy doll boy profoundly threatened new spaceman figure supplants jealous room top toy  '

In [45]:
movies.drop(columns = ['rated','writer','genre','director','actors','language','country','year','Key_words'], inplace=True)
movies.head()

,bag_of_words
title,
Toy Story,G johnlasseter animation adventure comedy f...
Jumanji,PG jonathanhensleigh adventure comedy family...
Grumpier Old Men,PG13 markstevenjohnson markstevenjohnson comed...
Waiting to Exhale,R terrymcmillan comedy drama romance forestw...
Father of the Bride Part II,PG alberthackett comedy family romance charl...


In [46]:
movies = movies.merge(temp, on='title')
movies.head()

,title,bag_of_words,weighted_rating,imdb_id
0,Toy Story,G johnlasseter animation adventure comedy f...,8.296249489578639,tt0114709
1,Jumanji,PG jonathanhensleigh adventure comedy family...,6.994508880344042,tt0113497
2,Grumpier Old Men,PG13 markstevenjohnson markstevenjohnson comed...,6.556503572029107,tt0113228
3,Waiting to Exhale,R terrymcmillan comedy drama romance forestw...,5.789027071723114,tt0114885
4,Father of the Bride Part II,PG alberthackett comedy family romance charl...,5.989632282691058,tt0113041


In [47]:
movies.set_index('title', inplace = True)

In [48]:
# Drop duplicate movies from the dataframe
movies = movies.drop_duplicates(subset='bag_of_words', keep= 'first')
movies.shape

(51520, 3)

In [49]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(movies['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(movies.index)
indices[:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [50]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.23732223, 0.15694121, ..., 0.06896552, 0.03052813,
        0.02417547],
       [0.23732223, 1.        , 0.12344268, ..., 0.06780635, 0.06003002,
        0.02376913],
       [0.15694121, 0.12344268, 1.        , ..., 0.03138824, 0.055577  ,
        0.06601783],
       ...,
       [0.06896552, 0.06780635, 0.03138824, ..., 1.        , 0.06105625,
        0.07252642],
       [0.03052813, 0.06003002, 0.055577  , ..., 0.06105625, 1.        ,
        0.10701463],
       [0.02417547, 0.02376913, 0.06601783, ..., 0.07252642, 0.10701463,
        1.        ]])

In [51]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_15_indexes = list(score_series.iloc[1:16].index)
    
    # populating the list with the titles of the best 15 matching movies
    for i in top_15_indexes:
        recommended_movies.append(list(movies.index)[i])
        
    return recommended_movies

In [80]:
def return_recs(title, orderby, df):
    title_a = recommendations(title)
    title_recs = df[df.index.isin(title_a)]
    print("Here are the movie recommendations for {}: ".format(title))
    if orderby == None:
        return title_recs[:10]
    else:
        return title_recs[:10].sort_values(by=[orderby], ascending=False)

In [53]:
Zootopia = recommendations('Zootopia')

In [54]:
Zootopia

['Moana',
 'Hoodwinked!',
 'Scooby-Doo! The Mystery Begins',
 'Ice Age: Collision Course',
 'Fantastic Mr. Fox',
 'Planet 51',
 'Bolt',
 'Sing',
 'Monsters, Inc.',
 'Trolls',
 'Happy Feet Two',
 'Who Framed Roger Rabbit',
 'Finding Dory',
 'Tangled Ever After',
 'Robots']

In [ ]:
Avatar = recommendations('Avatar')

In [55]:
Zootopia_recs = movies[movies.index.isin(Zootopia)]
Zootopia_recs[:10].sort_values(by=['weighted_rating'], ascending=False)

,bag_of_words,weighted_rating,imdb_id
title,,,
"Monsters, Inc.",G petedocter animation adventure comedy fam...,8.09627079384583,tt0198781
Fantastic Mr. Fox,PG roalddahl animation adventure comedy cri...,7.885822413468687,tt0432283
Who Framed Roger Rabbit,PG garyk.wolf animation adventure comedy cr...,7.686523757021259,tt0096438
Tangled Ever After,G nathangreno byronhoward markkennedy animatio...,7.559651885526486,tt2112281
Bolt,PG danfogelman animation adventure comedy d...,6.8921815347542,tt0397892
Hoodwinked!,PG coryedwards animation comedy crime famil...,6.483045098030427,tt0443536
Robots,PG ronmita animation adventure comedy famil...,6.294479408856629,tt0358082
Planet 51,PG joestillman animation adventure comedy f...,6.0907064307067476,tt0762125
Happy Feet Two,PG georgemiller garyeck warrencoleman paullivi...,5.894361638018661,tt1402488


In [ ]:
SpiritedAway = recommendations('Spirited Away')
Spirited_recs = movies[movies.index.isin(SpiritedAway)]
Spirited_recs[:10].sort_values(by=['weighted_rating'], ascending=False)

In [ ]:
Avatar = recommendations('Avatar')
avatar_recs = movies[movies.index.isin(Avatar)]
avatar_recs[:10].sort_values(by=['weighted_rating'], ascending=False)

In [ ]:
return_recs('Saw', 'weighted_rating')

In [ ]:
return_recs('Zootopia', 'weighted_rating')

In [ ]:
return_recs('The Princess Bride', 'weighted_rating')

In [56]:
test = pd.read_csv('/Users/user/final-capstone/inprogress3.csv')
test = test.drop_duplicates(subset='title', keep= 'first')


/Users/user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
test['imdb_votes'] = test['imdb_votes'].str.replace(",","")
test['imdb_votes'] = pd.to_numeric(test['imdb_votes'])


vote_counts = test[test['imdb_votes'].notnull()]['imdb_votes'].astype('int')
vote_averages = test[test['imdb_rating'].notnull()]['imdb_rating'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['imdb_votes']+1 # added +1 - Dan
    R = x['imdb_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

test['weighted_rating'] = test.apply(weighted_rating, axis=1)

In [58]:
test = test[['title','rated','plot','year','genre','director','actors','language','weighted_rating']]

In [60]:
test = test.set_index('title')

In [61]:
return_recs('Zootopia', 'weighted_rating', df = test)

Here are the movie recommendations for Zootopia: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
"Monsters, Inc.",G,"In order to power the city, monsters have to s...",2001,"Animation, Adventure, Comedy, Family, Fantasy","Pete Docter, David Silverman(co-director), Lee...","John Goodman, Billy Crystal, Mary Gibbs, Steve...","In order to power the city, monsters have to s...",English,8.098877
Fantastic Mr. Fox,PG,An urbane fox cannot resist returning to his f...,2009,"Animation, Adventure, Comedy, Crime, Drama, Fa...",Wes Anderson,"George Clooney, Meryl Streep, Jason Schwartzma...",An urbane fox cannot resist returning to his f...,"English, French",7.895716
Who Framed Roger Rabbit,PG,A toon-hating detective is a cartoon rabbit's ...,1988,"Animation, Adventure, Comedy, Crime, Family, F...",Robert Zemeckis,"Bob Hoskins, Christopher Lloyd, Joanna Cassidy...",A toon-hating detective is a cartoon rabbit's ...,English,7.695926
Tangled Ever After,G,"Rapunzel and Flynn Rider have their wedding, b...",2012,"Animation, Short, Action, Comedy, Family, Musical","Nathan Greno, Byron Howard","Mandy Moore, Zachary Levi, Alan Dale, Paul F. ...","Rapunzel and Flynn Rider have their wedding, b...",English,7.655589
Bolt,PG,The canine star of a fictional sci-fi/action s...,2008,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams","John Travolta, Miley Cyrus, Susie Essman, Mark...",The canine star of a fictional sci-fi/action s...,English,6.897634
Hoodwinked!,PG,"Little Red Riding Hood, the Wolf, the Woodsman...",2005,"Animation, Comedy, Crime, Family, Musical, Mys...","Cory Edwards, Todd Edwards(co-director), Tony ...","Anne Hathaway, Glenn Close, Jim Belushi, Patri...","Little Red Riding Hood, the Wolf, the Woodsman...","English, Spanish",6.494809
Robots,PG,"In a robot world, a young idealistic inventor ...",2005,"Animation, Adventure, Comedy, Family, Sci-Fi","Chris Wedge, Carlos Saldanha(co-director)","Paula Abdul, Halle Berry, Lucille Bliss, Terry...","In a robot world, a young idealistic inventor ...","English, Italian",6.298322
Planet 51,PG,An alien civilization is invaded by Astronaut ...,2009,"Animation, Adventure, Comedy, Family, Sci-Fi","Jorge Blanco, Javier Abad(co-director), Marcos...","Dwayne Johnson, Jessica Biel, Justin Long, Gar...",An alien civilization is invaded by Astronaut ...,"English, Spanish",6.097138
Happy Feet Two,PG,"Mumble's son, Erik, is struggling to realize h...",2011,"Animation, Adventure, Comedy, Family, Musical","George Miller, Gary Eck(co-director), David Pe...","Carlos Alazraqui, Lombardo Boyar, Jeffrey Garc...","Mumble's son, Erik, is struggling to realize h...",English,5.898242


In [62]:
return_recs('The Princess Bride', 'weighted_rating', df = test)

Here are the movie recommendations for The Princess Bride: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Harry Potter and the Sorcerer's Stone,PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",An orphaned boy enrolls in a school of wizardr...,English,7.598851
Where the Wild Things Are,PG,"Yearning for escape and adventure, a young boy...",2009,"Adventure, Drama, Family, Fantasy",Spike Jonze,"Max Records, Pepita Emmerichs, Max Pfeifer, Ma...","Yearning for escape and adventure, a young boy...",English,6.796106
Penelope,PG,A modern romantic tale about a young aristocra...,2006,"Comedy, Fantasy, Romance",Mark Palansky,"Richard E. Grant, Catherine O'Hara, Nick Pride...",A modern romantic tale about a young aristocra...,English,6.793465
The Witches,PG,A young boy stumbles onto a witch convention a...,1990,"Adventure, Comedy, Family, Fantasy, Horror, My...",Nicolas Roeg,"Anjelica Huston, Mai Zetterling, Jasen Fisher,...",A young boy stumbles onto a witch convention a...,English,6.788123
Tuck Everlasting,PG,A young woman meets and falls in love with a y...,2002,"Drama, Family, Fantasy, Romance",Jay Russell,"Alexis Bledel, William Hurt, Sissy Spacek, Jon...",A young woman meets and falls in love with a y...,"English, French",6.683123
Charlie and the Chocolate Factory,PG,A young boy wins a tour through the most magni...,2005,"Adventure, Comedy, Family, Fantasy, Musical",Tim Burton,"Johnny Depp, Freddie Highmore, David Kelly, He...",A young boy wins a tour through the most magni...,English,6.599209
The Three Musketeers,PG,A 90s retelling of the classic story.,1993,"Action, Adventure, Comedy, Family, Romance",Stephen Herek,"Charlie Sheen, Kiefer Sutherland, Chris O'Donn...",A 90s retelling of the classic story.,English,6.394894
Little Monsters,PG,A boy discovers an incredible and gruesome wor...,1989,"Adventure, Comedy, Family, Fantasy",Richard Greenberg,"Fred Savage, Howie Mandel, Daniel Stern, Marga...",A boy discovers an incredible and gruesome wor...,English,6.086765
The Jungle Book,PG,"Rudyard Kipling's classic tale of Mowgli, the ...",1994,"Adventure, Family, Romance",Stephen Sommers,"Jason Scott Lee, Cary Elwes, Lena Headey, Sam ...","Rudyard Kipling's classic tale of Mowgli, the ...",English,5.993388


In [73]:
return_recs('The Lord of the Rings: The Fellowship of the Ring', 'weighted_rating', df = test)

Here are the movie recommendations for The Lord of the Rings: The Fellowship of the Ring: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
The Lord of the Rings: The Return of the King,PG-13,Gandalf and Aragorn lead the World of Men agai...,2003,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston",Gandalf and Aragorn lead the World of Men agai...,"English, Quenya, Old English, Sindarin",8.899258
The Lord of the Rings: The Two Towers,PG,While Frodo and Sam edge closer to Mordor with...,2002,"Adventure, Drama, Fantasy",Peter Jackson,"Bruce Allpress, Sean Astin, John Bach, Sala Baker",While Frodo and Sam edge closer to Mordor with...,"English, Sindarin, Old English",8.699235
Harry Potter and the Deathly Hallows: Part 2,PG-13,"Harry, Ron, and Hermione search for Voldemort'...",2011,"Adventure, Drama, Fantasy, Mystery",David Yates,"Ralph Fiennes, Michael Gambon, Alan Rickman, D...","Harry, Ron, and Hermione search for Voldemort'...",English,8.098801
Thor,PG-13,The powerful but arrogant god Thor (Chris Hems...,2011,"Action, Adventure, Fantasy, Sci-Fi",Kenneth Branagh,"Chris Hemsworth, Natalie Portman, Tom Hiddlest...",The powerful but arrogant god Thor (Chris Hems...,English,6.999344
What Dreams May Come,PG-13,"Chris Nielsen dies in car crash one night, and...",1998,"Drama, Fantasy, Romance",Vincent Ward,"Robin Williams, Cuba Gooding Jr., Annabella Sc...","Chris Nielsen dies in car crash one night, and...",English,6.995326
The Lovely Bones,PG-13,Centers on a young girl who has been murdered ...,2009,"Drama, Fantasy, Thriller",Peter Jackson,"Mark Wahlberg, Rachel Weisz, Susan Sarandon, S...",Centers on a young girl who has been murdered ...,English,6.697651
The Long and Short of It,NaN,A workman gets some unexpected help from two p...,2003,"Short, Drama",Sean Astin,"Andrew Lesnie, Praphaphorn 'Fon' Chansantor, P...",A workman gets some unexpected help from two p...,English,6.642517
The Price of Milk,PG-13,New Zealand milk farmer Rob gives his lover Lu...,2000,"Comedy, Drama, Fantasy",Harry Sinclair,"Danielle Cormack, Karl Urban, Willa O'Neill, M...",New Zealand milk farmer Rob gives his lover Lu...,English,6.604069
Camille,PG-13,A twisted honeymoon adventure about a young co...,2008,"Adventure, Comedy, Drama, Fantasy, Romance",Gregory Mackenzie,"Sienna Miller, James Franco, David Carradine, ...",A twisted honeymoon adventure about a young co...,English,6.070842


In [64]:
return_recs('Spirited Away', 'weighted_rating', df = test)

Here are the movie recommendations for Spirited Away: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Howl's Moving Castle,PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",When an unconfident young woman is cursed with...,Japanese,8.196957
Whisper of the Heart,G,A love story between a girl who loves reading ...,1995,"Animation, Drama, Family, Music, Romance",Yoshifumi Kondô,"Yoko Honna, Issey Takahashi, Takashi Tachibana...",A love story between a girl who loves reading ...,"Japanese, English",7.980478
Porco Rosso,PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...","In 1930s Italy, a veteran World War I pilot is...",Japanese,7.788260
Ponyo,G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",A five-year-old boy develops a relationship wi...,Japanese,7.693614
Harry Potter and the Sorcerer's Stone,PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",An orphaned boy enrolls in a school of wizardr...,English,7.598851
Pom Poko,PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...",A community of magical shape-shifting raccoon ...,"Japanese, English",7.370404
The Cat Returns,G,"After helping a cat, a 17-year-old girl finds ...",2002,"Animation, Adventure, Comedy, Family, Fantasy,...",Hiroyuki Morita,"Chizuru Ikewaki, Yoshihiko Hakamada, Aki Maeda...","After helping a cat, a 17-year-old girl finds ...","Japanese, English",7.286123
My Neighbors the Yamadas,PG,The life and misadventures of a family in cont...,1999,"Animation, Comedy, Family",Isao Takahata,"Yukiji Asaoka, Tôru Masuoka, Masako Araki, Hay...",The life and misadventures of a family in cont...,Japanese,7.240213
Alice in Wonderland,PG,Alice (Tina Majorino) falls down a rabbit hole...,1999,"Adventure, Comedy, Family, Fantasy",Nick Willing,"Robbie Coltrane, Whoopi Goldberg, Ben Kingsley...",Alice (Tina Majorino) falls down a rabbit hole...,English,6.272653


In [65]:
return_recs('Titanic', 'weighted_rating', df = test)

Here are the movie recommendations for Titanic: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
The Notebook,PG-13,A poor yet passionate young man falls in love ...,2004,"Drama, Romance",Nick Cassavetes,"Tim Ivey, Gena Rowlands, Starletta DuPois, Jam...",A poor yet passionate young man falls in love ...,English,7.898402
500 Days of Summer,PG-13,An offbeat romantic comedy about a woman who d...,2009,"Comedy, Drama, Romance",Marc Webb,"Joseph Gordon-Levitt, Zooey Deschanel, Geoffre...",An offbeat romantic comedy about a woman who d...,"English, French, Swedish",7.698437
Ghosts of the Abyss,G,"James Cameron and Bill Paxton, director and ac...",2003,"Documentary, Family",James Cameron,"Bill Paxton, John Broadwater, Lori Johnston, C...","James Cameron and Bill Paxton, director and ac...",English,6.818386
Where the Heart Is,PG-13,A pregnant 17-year-old rebuilds her life after...,2000,"Comedy, Drama, Romance",Matt Williams,"Natalie Portman, Ashley Judd, Stockard Channin...",A pregnant 17-year-old rebuilds her life after...,English,6.787633
Crazy/Beautiful,PG-13,"At Pacific Palisades High, a poor Latino falls...",2001,"Drama, Romance",John Stockwell,"Kirsten Dunst, Jay Hernandez, Bruce Davison, H...","At Pacific Palisades High, a poor Latino falls...",English,6.488258
Dear John,PG-13,A romantic drama about a soldier who falls for...,2010,"Drama, Romance, War",Lasse Hallström,"Channing Tatum, Amanda Seyfried, Richard Jenki...",A romantic drama about a soldier who falls for...,English,6.298400
Tadpole,PG-13,Coming-of-age story about a suave 15-year-old ...,2002,"Comedy, Drama, Romance",Gary Winick,"Aaron Stanford, Kate Mara, Robert Iler, Peter ...",Coming-of-age story about a suave 15-year-old ...,"English, French",6.170935
In Love and War,PG-13,A bitter love story between a talented writer ...,1996,"Biography, Drama, Romance, War",Richard Attenborough,"Mackenzie Astin, Chris O'Donnell, Margot Stein...",A bitter love story between a talented writer ...,"English, Italian",5.987315
Daddy's Little Girls,PG-13,A mechanic falls in love with his attorney.,2007,"Drama, Romance",Tyler Perry,"Gabrielle Union, Idris Elba, Louis Gossett Jr....",A mechanic falls in love with his attorney.,English,5.605058


In [67]:
return_recs('Wings of Desire', 'weighted_rating', test)

Here are the movie recommendations for Wings of Desire: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Beauty and the Beast,G,A prince cursed to spend his days as a hideous...,1991,"Animation, Family, Fantasy, Musical, Romance","Gary Trousdale, Kirk Wise","Robby Benson, Jesse Corti, Rex Everhart, Angel...",A prince cursed to spend his days as a hideous...,"English, French",7.997970
The Science of Sleep,R,A man entranced by his dreams and imagination ...,2006,"Comedy, Drama, Fantasy, Romance",Michel Gondry,"Gael García Bernal, Charlotte Gainsbourg, Alai...",A man entranced by his dreams and imagination ...,"French, English, Spanish",7.291694
"Faraway, So Close!",PG-13,A group of angels in the German capital look l...,1993,"Drama, Fantasy, Romance",Wim Wenders,"Otto Sander, Peter Falk, Horst Buchholz, Mikha...",A group of angels in the German capital look l...,"German, French, English, Italian, Russian",7.222561
Silent Light,NaN,"In a Mennonite community in Mexico, a father's...",2007,"Drama, Romance",Carlos Reygadas,"Cornelio Wall, Miriam Toews, Maria Pankratz, P...","In a Mennonite community in Mexico, a father's...","Low German, Spanish, French, English",7.200728
Damage,R,A member of Parliament falls passionately in l...,1992,"Drama, Romance",Louis Malle,"Juliette Binoche, Jeremy Irons, Miranda Richar...",A member of Parliament falls passionately in l...,"English, French, German, Italian",6.872847
Water Drops on Burning Rocks,Not Rated,"In 1970s Germany, a 50-year-old businessman fa...",2000,"Comedy, Drama, Romance",François Ozon,"Bernard Giraudeau, Malik Zidi, Ludivine Sagnie...","In 1970s Germany, a 50-year-old businessman fa...","French, German",6.830298
City of Angels,PG-13,"Inspired by the modern classic, Wings of Desir...",1998,"Drama, Fantasy, Romance",Brad Silberling,"Nicolas Cage, Meg Ryan, Andre Braugher, Dennis...","Inspired by the modern classic, Wings of Desir...",English,6.696798
Voyager,PG-13,Walter Faber has survived a crash with an airp...,1991,"Drama, Romance",Volker Schlöndorff,"Sam Shepard, Julie Delpy, Barbara Sukowa, Diet...",Walter Faber has survived a crash with an airp...,"German, English, Spanish, French, Greek",6.588390
Land of Plenty,NOT RATED,A drama that investigates anxiety and disillus...,2004,"Drama, War",Wim Wenders,"Michelle Williams, John Diehl, Shaun Toub, Wen...",A drama that investigates anxiety and disillus...,English,6.429876


In [69]:
return_recs('Princess Mononoke', 'weighted_rating', test)

Here are the movie recommendations for Princess Mononoke: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Spirited Away,PG,"During her family's move to the suburbs, a sul...",2001,"Animation, Adventure, Family, Fantasy, Mystery","Hayao Miyazaki, Kirk Wise","Rumi Hiiragi, Miyu Irino, Mari Natsuki, Takash...","During her family's move to the suburbs, a sul...","Japanese, English",8.598195
Howl's Moving Castle,PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",When an unconfident young woman is cursed with...,Japanese,8.196957
Porco Rosso,PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...","In 1930s Italy, a veteran World War I pilot is...",Japanese,7.788260
The Wind Rises,PG-13,"A look at the life of Jiro Horikoshi, the man ...",2013,"Animation, Biography, Drama, History, Romance,...",Hayao Miyazaki,"Hideaki Anno, Hidetoshi Nishijima, Miori Takim...","A look at the life of Jiro Horikoshi, the man ...","Japanese, German, Italian, French",7.788136
Ponyo,G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",A five-year-old boy develops a relationship wi...,Japanese,7.693614
Pom Poko,PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...",A community of magical shape-shifting raccoon ...,"Japanese, English",7.370404
Street Fighter II: The Animated Movie,PG-13,"Bison, the ruthless leader of the internationa...",1994,"Animation, Action, Adventure, Fantasy, Sci-Fi",Gisaburô Sugii,"Kôjiro Shimizu, Kenji Haga, Miki Fujitani, Mas...","Bison, the ruthless leader of the internationa...",Japanese,7.228503
Steamboy,PG-13,"In 1860s Britain, a boy inventor finds himself...",2004,"Animation, Action, Adventure, Fantasy, Sci-Fi,...",Katsuhiro Ôtomo,"Anne Suzuki, Masane Tsukayama, Katsuo Nakamura...","In 1860s Britain, a boy inventor finds himself...","Japanese, English",6.874282
Tales from Earthsea,PG-13,"In a mythical land, a man and a young boy inve...",2006,"Animation, Adventure, Fantasy",Gorô Miyazaki,"Jun'ichi Okada, Aoi Teshima, Bunta Sugawara, Y...","In a mythical land, a man and a young boy inve...","Japanese, English",6.490570


In [71]:
return_recs('Amélie', 'weighted_rating', test)

Here are the movie recommendations for Amélie: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Delicatessen,R,Post-apocalyptic surrealist black comedy about...,1991,"Comedy, Crime","Marc Caro, Jean-Pierre Jeunet","Pascal Benezech, Dominique Pinon, Marie-Laure ...",Post-apocalyptic surrealist black comedy about...,French,7.690886
Rendezvous in Paris,NaN,Three stories of love and coincidence around t...,1995,"Comedy, Romance",Éric Rohmer,"Clara Bellar, Antoine Basler, Mathias Mégard, ...",Three stories of love and coincidence around t...,French,7.144882
Private Fears in Public Places,NaN,"In Paris, six people all look for love, despit...",2006,"Drama, Romance",Alain Resnais,"Sabine Azéma, Isabelle Carré, Laura Morante, P...","In Paris, six people all look for love, despit...","French, English",6.886018
Romantics Anonymous,PG-13,What happens when a man and a woman share a co...,2010,"Comedy, Romance",Jean-Pierre Améris,"Benoît Poelvoorde, Isabelle Carré, Lorella Cra...",What happens when a man and a woman share a co...,French,6.859491
Brief Crossing,Unrated,A young French man and an older English woman ...,2001,"Comedy, Drama, Romance",Catherine Breillat,"Sarah Pratt, Gilles Guillain, Marc Filipi, Laë...",A young French man and an older English woman ...,"French, English",6.673728
I Can't Sleep,Unrated,Beautiful Daiga has emigrated from Lithuania t...,1994,"Drama, Mystery, Romance, Thriller",Claire Denis,"Yekaterina Golubeva, Richard Courcet, Vincent ...",Beautiful Daiga has emigrated from Lithuania t...,"French, Russian, English",6.666429
27 Missing Kisses,NaN,"The summerly adventures of teen Sybilla, who f...",2000,"Comedy, Drama, Romance",Nana Dzhordzhadze,"Nutsa Kukhianidze, Evgeniy Sidikhin, Shalva Ia...","The summerly adventures of teen Sybilla, who f...","Georgian, Russian, French, English",6.567521
Beautiful Lies,NaN,A hairdresser forwards a passionate love lette...,2010,"Comedy, Romance",Pierre Salvadori,"Audrey Tautou, Nathalie Baye, Sami Bouajila, S...",A hairdresser forwards a passionate love lette...,"French, Chinese, English",6.460667
Mr. Bean's Holiday,G,Mr. Bean wins a trip to Cannes where he unwitt...,2007,"Comedy, Family",Steve Bendelack,"Rowan Atkinson, Steve Pemberton, Lily Atkinson...",Mr. Bean wins a trip to Cannes where he unwitt...,"English, French, Spanish, Russian",6.397741


In [89]:
return_recs('The Dictator','weighted_rating', test)

Here are the movie recommendations for The Dictator: 


,rated,plot,year,genre,director,actors,plot,language,weighted_rating
title,,,,,,,,,
Bill Burr: You People Are All the Same.,NaN,Comic Bill Burr's 2012 comedy special.,2012,"Documentary, Comedy",Jay Karas,"Bill Burr, Paul Virzi",Comic Bill Burr's 2012 comedy special.,English,8.117330
Religulous,R,Bill Maher's take on the current state of worl...,2008,"Documentary, Comedy, War",Larry Charles,"Bill Maher, Tal Bachman, Jonathan Boulden, Ste...",Bill Maher's take on the current state of worl...,"English, Hebrew, Arabic, Persian, Spanish",7.687827
Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan,R,Kazakh TV talking head Borat is dispatched to ...,2006,Comedy,Larry Charles,"Sacha Baron Cohen, Ken Davitian, Luenell, Chester",Kazakh TV talking head Borat is dispatched to ...,"English, Romanian, Hebrew, Polish, Armenian",7.298349
EuroTrip,R,"Dumped by his girlfriend, a high school grad d...",2004,"Adventure, Comedy","Jeff Schaffer, Alec Berg(co-director), David M...","Scott Mechlowicz, Jacob Pitts, Kristin Kreuk, ...","Dumped by his girlfriend, a high school grad d...","English, French, German, Italian, Japanese",6.598309
In Montauk,NaN,The hardest choices in life are the ones we th...,2012,"Drama, Romance",Kim Cummings,"Nina Kaczorowski, Lukas Hassel, George Katt, C...",The hardest choices in life are the ones we th...,English,6.435251
Fancypants,PG-13,The story of a professional wrestler who's afr...,2011,"Comedy, Drama",Joshua Russell,"Patrick Gleason, Roddy Piper, Robert Carradine...",The story of a professional wrestler who's afr...,English,5.925171
Brüno,R,Flamboyant and gay Austrian Brüno looks for ne...,2009,Comedy,Larry Charles,"Sacha Baron Cohen, Gustaf Hammarsten, Clifford...",Flamboyant and gay Austrian Brüno looks for ne...,"English, German",5.799751
16-Love,PG,When the number one junior player in the count...,2012,"Comedy, Family, Romance",Adam Lipsius,"Lindsey Shaw, Chandler Massey, Keith Coulouris...",When the number one junior player in the count...,English,5.781777
Skittin Across America,NaN,What would you do to make your dreams come tru...,2009,Comedy,Shane Hartline,"David Bottaro, Garrett Bryant, Joseph Frantz, ...",What would you do to make your dreams come tru...,English,5.710939


# Key Findings

1. This is a good initial filter for a website or app that gives movie recommendations for movies that are similar
2. Depending on the movie, key metrics like diveristy/variety differ. Below is analysis for a few of the recommendations:

## 1. The Lord of the Rings: The Fellowship of the Ring
All the movies that are also in the Lord of the Rings Franchise are returned, along with Harry Potter. But it also suggests a variety of other movies that are somewhat similar in tone, though none of them seem to be High Fantasy the same way that Lord of the Rings is. It's important to note, however, that there are two filters: The first filters for cosine similarity, the second filters the results by the weighted IMDB rating. By cosine similarity **only**, we also get movies from the Hobbit franchise, which are closely related to Lord of the Rings as it exists in the same Universe. However, these movies had a lower weighted rating on IMDB than other movies, so were suggested later. 

Movies recommended **ordered by cosine similarity**:

1. **'The Lord of the Rings: The Return of the King'**, also in the franchise
2. **'The Lord of the Rings: The Two Towers'**, also in the franchise
3. **'Harry Potter and the Deathly Hallows: Part 2'** a similar movie in terms of high fantasy and plot
4. **'The Hobbit: The Desolation of Smaug'** also in the franchise
5. **'The Hobbit: An Unexpected Journey'** also in the franchise
6. **'The Long and Short of It'** - a short film made by crew during the filming of LoTR: The Two Towers
7. **'What Dreams May Come'** - a 1998 film that features tangential similarities, a 'journey' through a foreign land as the main character traverses through the afterlife in various forms, I'm not sure if this would be popular for lovers of LoTR as it's a different kind of 'fantasy' and 'drama'
8. **'The Hobbit: The Battle of the Five Armies'** another Hobbit film
9. **'The Lovely Bones'** - Another movie that involves the afterlife, I'm not sure this would be popular among people who love LoTR
10. **'The Price of Milk'** - A romantic comedy filmed in New Zealand, where the LoTR franchise was filmed
11. **'Thor'** - probably more of a movie that would be enjoyed by LoTR fans, as it's fantasy and deals more with 'war'
12. **'Hunt for the Wilderpeople'** - Another example of I think the algorithm using 'drama'
13. **'Captain Marvel'** - Similar to Thor, I think this is closer to what users would want to be recommended
14. **'Joe Dirt'** - similar tale of a hero's journey, though I think it misses the mark
15. **'Camille'** - has to do with journey through the afterlife

### Overall takeaway for LoTR: Fellowship Recommendations

The recommendations all have to do with a hero's journey, loss, and in many cases the afterlife. Most of them have some sort of fantastical element, though the recommendations fall short of what LoTR fans might like. I don't think this is a great filter to show to people who love recommendations, but think it could be improved if there were some more keywords (perhaps user-inputted keywords like 'High fantasy' and 'war'). 

The MovieLens recommender also recommends Star Wars and Indiana Jones based on Lord of the Rings. For computational time, this version of the filter only uses movies from 1988 and beyond, and therefore misses Star Wars and Indiana Jones, for the most part, but not the new Star Wars movies, which I suppose are not close enough to be recommended.

It's important to note that, in terms of themes, these **are good picks**. These movies all have thematic links, about loss and sacrifice and the inevitability of entropy and how power attracts the corruptible. Except Joe Dirt, I'm not sure what that's about.



## 2. Spirited Away
The results for *Spirited Away*, the award-winning 2001 animation, are a little more consistent across the board. Most of the other movies are also Japanese animation from the same studio, Studio Ghibli, making the diversity and variety of this set lower than Lord of the Rings, which recommended quite a few different genres, directors, and film ideas (which is good and bad, in its own way).

Movies recommended **filtered by cosine similarity and ordered by the weighted IMDB rating**:

* **Howl's Moving Castle** - deals with many similar themes and is from the same studio/director
* **Whisper of the Heart** - Also from studio Ghibli
* **Porco Rosso** - Also from Miyazaki
* **Ponyo** - Also from Miyazaki
* **Harry Potter and the Sorcerer's Stone** - a similar theme and idea of a child taken away from their parents and treated poorly by those around them (Harry loses his parents, Chihiro in Spirited Away has her parents turned into pigs). Other, magical themes run through both of these stories, making this a decent recommendation.
* **Pom Poko** - from Studio Ghibli / Miyazaki
* **The Cat Returns** - from Studio Ghibli / Miyazaki
* **My Neighbors the Yamadas** - from Studio Ghibli / Miyazaki
* **Alice in Wonderland** - I really like that Alice in Wonderland is recommended, as the stories have similarities. Both feature a strong female lead, a young girl who gets lost in a world that isn't her own and has to find her way home.
* **Rugrats Go Wild** - About a bunch of kids who get lost in the woods, and is animation, so I can see it, although this is where the personalization comes in, because the brunt metadata is very similar but this isn't a great recommendation

### Overall takeway for Spirited Away:
These recommendations are certainly more consistent and better than they were for Lord of Rings, but are a little strict in terms of who the director is, for example. I would have loved to get back some other Japanese animation (however I don't even know if that's in my dataframe). However, again, the algorithm has returned movies that rest on pretty similar thematic lines.

## Overall Findings

The algorithm is good at finding underlying thematic tones of movies and returning them, even if they aren't strictly in the same genre. Some tuning to the algorithm would perhaps enhance it, playing with NLP to see what could be done with n-grams instead of just a bag of words, or other processes like word2vec